# Estimate the Pumping of a Pumped Hydro Energy Storage

This notebook estimates the pumping of a pumped hydro energy storage (PHES) using time series of production data. This data is provided by Energy Quantified and fetched via their Python client.

Many PHES plants are missing pumping data and only provide data on production. The aim is to estimate the pumping from the production and other external time series. Two plants in Europe have been identified with good quality in their data to analyze the results. These are the Kruonis plant in Lithuania and the Čierny Váh plant in Slovakia.

| Plant | LT Kruonius | SK Čierny Váh |
|---|---|---|
| Max. Capacity | 900 MW | 735.16 MW |
| Efficiency | 74 % | 73 % |
| Max. Storage | 10,800 MWh | 4,000 MWh | 

In [1]:
from datetime import date
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression

from energyquantified import EnergyQuantified 
from energyquantified.time import CET

# set print options to print all rows
pd.set_option('display.max_rows', None)
# set sns figure size
sns.set_theme(rc={'figure.figsize':(15, 7)})

In [ ]:
# setup EQ's Python client
eq = EnergyQuantified(api_key_file="../eq_api_key.txt")
eq.is_api_key_valid()

This notebook uses the LT Kruonis plant but it can be easily switched to SK Čierny Váh by commenting and uncommenting the following lines.

In [ ]:
area = "LT"
plant_name = "LT @Kruonis"
plant_efficiency = 0.74
max_storage = 10_800
max_capacity = 900
capacity_steps = [-225, -450, -675, -900]
spot_price_curve = "LT Price Spot EUR/MWh NordPool H Actual"
begin_date = date(2021, 4, 12) # first date with residual productin data
end_date = date(2024, 7, 1)

# area = "SK"
# plant_name = "SK @Cierny-Vah"
# plant_efficiency = 0.73
# max_storage = 4_000
# max_capacity = 400
# capacity_steps = [-100, -200, -300, -400, -500, -600]
# spot_price_curve = "SK Price Spot EUR/MWh OKTE H Actual"
# begin_date = date(2018, 11, 1) # first date with residual productin data
# end_date = date(2024, 7, 1)

## Load Data

In [ ]:
plant_ts = eq.timeseries.load(f"{plant_name} Hydro Pumped-storage Production MWh/h H Actual", begin=begin_date, end=end_date)
spot_price_ts = eq.timeseries.load(spot_price_curve, begin=begin_date, end=end_date)
resid_load_ts = eq.timeseries.load(f"{area} Residual Load MWh/h H Actual", begin=begin_date, end=end_date)
resid_prod_ts = eq.timeseries.load(f"{area} Residual Production Day-Ahead MWh/h H Synthetic", begin=begin_date, end=end_date)

# create data frame
df = plant_ts.to_df(name="Net Production", single_level_header=True).fillna(0)
df["Production"] = df["Net Production"].clip(lower=0)
df["Pumping"] = df["Net Production"].clip(upper=0)

df["Spot Price"] = spot_price_ts.to_df(name="spot_price", single_level_header=True)["spot_price"]
df["Residual Load"] = resid_load_ts.to_df(name="resid_load", single_level_header=True)["resid_load"]
df["Residual Production"] = resid_prod_ts.to_df(name="resid_prod", single_level_header=True)["resid_prod"]

df['Residual Production'] = df['Residual Production'].interpolate(method='linear')

df.info()

In [ ]:
df.describe()

# Analyse Data

## Time

In [ ]:
# extract time features
df["hour"] = df.index.hour
df["weekday"] = df.index.weekday
df["month"] = df.index.month
df["year"] = df.index.year

In [ ]:
time_features = ["hour", "weekday", "month", "year"]
production_features = ["Net Production", "Production", "Pumping"]
corr_matrix = df.corr()
corr_subset = corr_matrix.loc[time_features, production_features]
corr_subset

In [ ]:
sns.heatmap(corr_subset, annot=True, vmin=-1, vmax=1, cmap="coolwarm", fmt=".2f")
plt.title("Correlation matrix between time features and production features")
plt.ylabel("Time features")
plt.xlabel("Production features")
plt.show()

In [ ]:
plt.figure(figsize=(15, 3))
sns.histplot(df[["Pumping", "Production"]].replace(0, np.nan), kde=True)
# plt.title(f"{plant_name} production and pumping distribution")
plt.xlabel("MWh/h")
plt.show()

In [ ]:
prod_pump_df = df["Net Production"].replace(0, np.nan).gt(0).replace({True: "Production", False: "Pumping"})
plt.figure(figsize=(15, 5))
sns.boxenplot(
    data=df,
    y="Net Production",
    x="hour",
    showfliers=False,
    hue=prod_pump_df
)
# plt.title(f"{plant_name} Production and Pumping by Hour")
plt.ylabel("MWh/h")
plt.xlabel("Hour of Day")
# plt.legend(title="Production / Pumping")
plt.show()

## Correlation

In [ ]:
sns.histplot(
    df["Spot Price"],
    log_scale=True
)
plt.title(f"{area} spot price distribution")
plt.xlabel("€")
plt.show()

In [ ]:
sns.histplot(df[["Residual Production", "Residual Load"]])
plt.title(f"{area} Residual Load and Production distribution")
plt.xlabel("MWh/h")
plt.show()

In [ ]:
df.corr()

In [ ]:
# set sns figure size
sns.set_theme(rc={'figure.figsize':(6, 4)})

sns.heatmap(
    df[["Spot Price" ,"Residual Load" ,"Residual Production" ,"Net Production" ,"Production" ,"Pumping"]]
        .corr()
        .loc[["Net Production", "Production", "Pumping"], ["Spot Price", "Residual Load", "Residual Production"]],
    vmin=-1,
    vmax=1,
    annot=True,
    cmap='coolwarm',
    fmt=".2f",
    # set font size
    annot_kws={"size": 18}
)
# plt.title(f"Correlation matrix between {area} and {plant_name} production features")
# plt.ylabel(f"PHS plant")
# plt.xlabel(f"Area")

plt.show()

In [ ]:
sns.scatterplot(
    df,
    x="Residual Production",
    y="Net Production",
    hue="Spot Price",
    hue_norm=(0, 500),
)
plt.ylabel("Net Production [MWh/h]")
plt.xlabel("Residual Production [MWh/h]")
plt.show()

In [ ]:
sns.regplot(
    df,
    x="Residual Production",
    y="Net Production",
    # hue="Spot Price",
    # hue_norm=(0, 500),
    line_kws={'color': 'orange'}
)
plt.ylabel("Net Production [MWh/h]")
plt.xlabel("Residual Production [MWh/h]")
plt.show()

In [ ]:
g = sns.JointGrid(
    df.resample("6h").mean(),
    x="Residual Production",
    y="Net Production",
    # hue="Spot Price",
    # hue_norm=(0, 500),
    # kind="reg",
)
g.plot_joint(sns.regplot, line_kws={'color': 'orange'})
# g.plot_marginals(sns.boxenplot, showfliers=False)
g.plot_marginals(sns.histplot, kde=True)
plt.ylabel("Net Production [MWh/h]")
plt.xlabel("Residual Production [MWh/h]")
plt.show()

In [ ]:
# set sns figure size
sns.set_theme(rc={'figure.figsize':(15, 7)})
sns.lineplot(df[["Residual Production", "Net Production"]].tail(14*24), dashes=False)

# Linear Regression

In [ ]:
def adjust_pumping_to_production(prod: pd.Series, pump: pd.Series, plant_efficiency: float, window_size: int) -> pd.Series:
    # calculate sum of production and pumping over a period of window_size
    pump_ms: pd.Series = pump.rolling(window_size).sum()
    prod_ms: pd.Series = prod.rolling(window_size).sum()

    # calculate factor to scale estimated pumping to fit amount of production
    pump_scale_factor: float = prod_ms / (-plant_efficiency * pump_ms)
    
    return pump * pump_scale_factor

def adjust_pumping_to_capacities(net_prod_est: pd.Series, capacity_steps: list[int]) -> pd.Series:
    # Add 0 as the first step because its a valid capacity
    capacity_steps.insert(0, 0)
    # Round the estimated net production to the nearest capacity step
    return net_prod_est.apply(lambda x: x if pd.isna(x) or x >= 0 else min(capacity_steps, key=lambda y: abs(y - x)))

def combine_estimated_adjusted(net_prod_est: pd.Series, net_prod_adj: pd.Series) -> pd.Series:
    return (net_prod_est + net_prod_adj) / 2.0

def estimate_pumping(net_prod: pd.Series, resid_prod: pd.Series, resid_load: pd.Series, spot_price: pd.Series, window_size_ma: int, window_size_ms: int, plant_efficiency: float) -> tuple[pd.Series, pd.Series, pd.Series, pd.Series, list[float], float]:
    # build data frame
    df_cp = pd.DataFrame({
        'Net Production': net_prod.fillna(0),
        'Net Production MA': net_prod.rolling(window_size_ma).mean(),
        'Net Production Std': net_prod.rolling(window_size_ma).std(),
        'Production': net_prod.clip(lower=0),
        'Production MA': net_prod.clip(lower=0).rolling(window_size_ma).mean(),
        'Production Std': net_prod.clip(lower=0).rolling(window_size_ma).std(),
        'Pumping': net_prod.clip(upper=0),
        'Pumping MA': net_prod.clip(upper=0).rolling(window_size_ma).mean(),
        'Pumping Std': net_prod.clip(upper=0).rolling(window_size_ma).std(),
        'Residual Production': resid_prod,
        'Residual Production MA': resid_prod.rolling(window_size_ma).mean(),
        'Residual Production Std': resid_prod.rolling(window_size_ma).std(),
        'Residual Load': resid_load,
        'Residual Load MA': resid_load.rolling(window_size_ma).mean(),
        'Residual Load Std': resid_load.rolling(window_size_ma).std(),
        'Spot Price': spot_price,
        'Spot Price MA': spot_price.rolling(window_size_ma).mean(),
        'Spot Price Std': spot_price.rolling(window_size_ma).std(),
        })

    # Normalize the input features    
    df_cp['Net Production Normalized'] = ((df_cp['Net Production'] - df_cp['Net Production MA']) / df_cp['Net Production Std']).fillna(0)
    df_cp['Residual Production Normalized'] = ((df_cp['Residual Production'] - df_cp['Residual Production MA']) / df_cp['Residual Production Std']).fillna(0)
    df_cp['Residual Load Normalized'] = ((df_cp['Residual Load'] - df_cp['Residual Load MA']) / df_cp['Residual Load Std']).fillna(0)
    df_cp['Spot Price Normalized'] = ((df_cp['Spot Price'] - df_cp['Spot Price MA']) / df_cp['Spot Price Std']).fillna(0)

    # Define the input features (X) and the target variable (y)
    X: pd.DataFrame = df_cp[['Residual Production Normalized', 'Residual Load Normalized', 'Spot Price Normalized']].iloc[window_size_ma:]
    y: pd.Series = df_cp['Net Production Normalized'].iloc[window_size_ma:]

    # Train the Linear Regression model
    model: LinearRegression = LinearRegression().fit(X, y)

    # Predict the output for all time points
    net_prod_est_zero_arr: np.ndarray = model.predict(X)
    net_prod_est_zero: pd.Series = pd.Series(net_prod_est_zero_arr, index=y.index)
    
    # STAGE 1 Estimation: Linear Regression With Adjustment of Production-Pumping-Baalance
    net_prod_est_one: pd.Series = df_cp['Production'] + adjust_pumping_to_production(df_cp['Production'], net_prod_est_zero.clip(upper=0), plant_efficiency, window_size_ms)

    # STAGE 2 Estimation: Capacity Steps
    net_prod_est_two: pd.Series = adjust_pumping_to_capacities(net_prod_est_one, capacity_steps)

    # STAGE 3 Estimation: Combine Estimated and Adjusted
    net_prod_est_three: pd.Series = combine_estimated_adjusted(net_prod_est_one, net_prod_est_two)

    # STAGE 4 Estimation: Readjustment of Production-Pumping-Balance
    net_prod_est_four: pd.Series = df_cp['Production'] + adjust_pumping_to_production(df_cp['Production'], net_prod_est_three.clip(upper=0), plant_efficiency, window_size_ms)

    return (net_prod_est_one, net_prod_est_two, net_prod_est_three, net_prod_est_four, model.coef_, model.intercept_)

In [ ]:
def calculate_stats(net_prod: pd.Series, net_prod_est: pd.Series) -> tuple[float, float, float, float, float, float, float, float, float]:
    mae = (net_prod - net_prod_est).abs().mean()
    mse = ((net_prod - net_prod_est) ** 2).mean()
    mape = (net_prod - net_prod_est).abs().sum() / net_prod.abs().sum()
    rmse = np.sqrt(((net_prod - net_prod_est) ** 2).mean())
    cum_sum = (net_prod - net_prod_est).sum()
    median = (net_prod - net_prod_est).abs().median()
    std = (net_prod - net_prod_est).std()
    skew = (net_prod - net_prod_est).skew()
    kurt = (net_prod - net_prod_est).kurt()

    return (mae, mse, mape, rmse, cum_sum, median, std, skew, kurt)

In [ ]:
steps = [1, 2, 3, 5, 7, 10, 14, 30, 60, 90, 180, 270, 360]
results = []
for window_size_ma in steps:
    for window_size_ms in steps:
# for window_size in range(1, 365):
#     for window_size_ms in range(1, 365):
        print(f"Window Size MA: {window_size_ma}\t\tWindow Size MS: {window_size_ms} ")
        net_prod_est_one, net_prod_est_two, net_prod_est_three, net_prod_est_four, coefficients, intercept = estimate_pumping(df['Net Production'], df['Residual Production'], df['Residual Load'], df['Spot Price'], window_size_ma=window_size_ma * 24, window_size_ms=window_size_ms * 24, plant_efficiency=plant_efficiency)
        stats_est = calculate_stats(df['Net Production'], net_prod_est_four)
        results.append((window_size_ma, window_size_ms, window_size_ms, coefficients, intercept, *stats_est, net_prod_est_four))
print(f"Found {len(results)} results")

In [ ]:
def print_top_ten(results, sort_idx: int):
    results.sort(key=lambda x: x[sort_idx])
    # print top ten results
    for idx, result in enumerate(results):
        print(f"{idx + 1}: Window Size MA: {result[0]}\tWindow Size MS: {result[1]}\tWindow Size MS 2: {result[2]}\tCoefficients: {result[3]}\tIntercept: {result[4]:.5f}\tMAE: {result[5]:.2f}\tMSE: {result[6]:.2f}\tMAPE: {result[7]*100:.0f}%\tRMSE: {result[8]:.2f}\tCumSum: {result[9]:.2f}\tMedian: {result[10]:.2f}\tStd: {result[11]:.2f}\tSkew: {result[12]:.2f}\tKurt: {result[13]:.2f}")
        if idx == 9:
            break

print("\nTop results by MAE")
print_top_ten(results, 5)

print("\nTop results by MAPE")
print_top_ten(results, 7)

print("\nTop results by RMSE")
print_top_ten(results, 8)

print("\nTop results by CumSum")
print_top_ten(results, 9)

In [ ]:
stats_df = pd.DataFrame(results, columns=["Window Size MA", "Window Size MS", "Window Size MS 2", "Coefficients", "Intercept", "MAE", "MSE", "MAPE", "RMSE", "CumSum", "Median", "Std", "Skew", "Kurt", "Data"])
stats_df.describe()

In [ ]:
plt.figure(figsize=(15, 5))
sns.swarmplot(
    stats_df,
    hue="Window Size MS",
    y="RMSE",
    x="Window Size MA",
    hue_order=steps,
    size=10,
)
plt.xlabel("Window Size of Moving Average in Days")
plt.ylabel("Root Mean Squared Error")
plt.legend(title="Window Size of\nMoving Sum\nin Days", loc='upper left')
plt.ylim(60, 150)
# adjust the labels shown in the legend

plt.show()

In [ ]:
# LT @Kruonis
window_size_ma = 14
window_size_ms = 90

# SK @Cierny-Vah
window_size = 1
window_size_ms = 90

net_prod_est_one, net_prod_est_two, net_prod_est_three, net_prod_est_four, coefficients, intercept = estimate_pumping(df['Net Production'], df['Residual Production'], df['Residual Load'], df['Spot Price'], window_size_ma=window_size_ma*24, window_size_ms=window_size_ms*24, plant_efficiency=plant_efficiency)
stats_est_one = calculate_stats(df['Net Production'], net_prod_est_one)
stats_est_two = calculate_stats(df['Net Production'], net_prod_est_two)
stats_est_three = calculate_stats(df['Net Production'], net_prod_est_three)
stats_est_four = calculate_stats(df['Net Production'], net_prod_est_four)

diff_one = df['Net Production'] - net_prod_est_one
diff_two = df['Net Production'] - net_prod_est_two
diff_three = df['Net Production'] - net_prod_est_three
diff_four = df['Net Production'] - net_prod_est_four

# print both, stats_est and stats_est_two, as table
print(f"""
Intercept: {intercept}
Coefficients: {coefficients}

\t\tEstimated\tAdjusted\tCombined\tReadjusted
MAE:\t\t{stats_est_one[0]:.2f}\t\t{stats_est_two[0]:.2f}\t\t{stats_est_three[0]:.2f}\t\t{stats_est_four[0]:.2f}
MSE:\t\t{stats_est_one[1]:.0f}\t\t{stats_est_two[1]:.0f}\t\t{stats_est_three[1]:.0f}\t\t{stats_est_four[1]:.0f}
MAPE:\t\t{stats_est_one[2]*100:.0f}%\t\t{stats_est_two[2]*100:.0f}%\t\t{stats_est_three[2]*100:.0f}%\t\t{stats_est_four[2]*100:.0f}%
RMSE:\t\t{stats_est_one[3]:.2f}\t\t{stats_est_two[3]:.2f}\t\t{stats_est_three[3]:.2f}\t\t{stats_est_four[3]:.2f}
CumSum:\t\t{stats_est_one[4]:.0f}\t\t{stats_est_two[4]:.0f}\t\t{stats_est_three[4]:.0f}\t\t{stats_est_four[4]:.0f}
Median:\t\t{stats_est_one[5]:.2f}\t\t{stats_est_two[5]:.2f}\t\t{stats_est_three[5]:.2f}\t\t{stats_est_four[5]:.2f}
Std:\t\t{stats_est_one[6]:.2f}\t\t{stats_est_two[6]:.2f}\t\t{stats_est_three[6]:.2f}\t\t{stats_est_four[6]:.2f}
Skew:\t\t{stats_est_one[7]:.2f}\t\t{stats_est_two[7]:.2f}\t\t{stats_est_three[7]:.2f}\t\t{stats_est_four[7]:.2f}
Kurt:\t\t{stats_est_one[8]:.2f}\t\t{stats_est_two[8]:.2f}\t\t{stats_est_three[8]:.2f}\t\t{stats_est_four[8]:.2f}
""")

print(f"""
"MAE", "{stats_est_one[0]:.2f}", "{stats_est_two[0]:.2f}", "{stats_est_three[0]:.2f}", "{stats_est_four[0]:.2f}",
"MSE", "{stats_est_one[1]:.0f}", "{stats_est_two[1]:.0f}", "{stats_est_three[1]:.0f}", "{stats_est_four[1]:.0f}",
"MAPE", "{stats_est_one[2]*100:.0f}%", "{stats_est_two[2]*100:.0f}%", "{stats_est_three[2]*100:.0f}%", "{stats_est_four[2]*100:.0f}%",
"RMSE", "{stats_est_one[3]:.2f}", "{stats_est_two[3]:.2f}", "{stats_est_three[3]:.2f}", "{stats_est_four[3]:.2f}",
"Cum. Sum", "{stats_est_one[4]:,.0f}", "{stats_est_two[4]:,.0f}", "{stats_est_three[4]:,.0f}", "{stats_est_four[4]:,.0f}",
"Median", "{stats_est_one[5]:.2f}", "{stats_est_two[5]:.2f}", "{stats_est_three[5]:.2f}", "{stats_est_four[5]:.2f}",
"Std. Dev.", "{stats_est_one[6]:.2f}", "{stats_est_two[6]:.2f}", "{stats_est_three[6]:.2f}", "{stats_est_four[6]:.2f}",
"Skewness", "{stats_est_one[7]:.2f}", "{stats_est_two[7]:.2f}", "{stats_est_three[7]:.2f}", "{stats_est_four[7]:.2f}",
"Kurtosis", "{stats_est_one[8]:.2f}", "{stats_est_two[8]:.2f}", "{stats_est_three[8]:.2f}", "{stats_est_four[8]:.2f}",
      """)

df['Net Production Estimated One'] = net_prod_est_one
df['Net Production Estimated Two'] = net_prod_est_two
df['Net Production Estimated Three'] = net_prod_est_three
df['Net Production Estimated Four'] = net_prod_est_four
df['Net Production Difference One'] = diff_one
df['Net Production Difference Two'] = diff_two
df['Net Production Difference Three'] = diff_three
df['Net Production Difference Four'] = diff_four

df[['Net Production', 'Net Production Estimated One', 'Net Production Estimated Two', 'Net Production Estimated Three', 'Net Production Estimated Four', 'Net Production Difference One', 'Net Production Difference Two', 'Net Production Difference Three', 'Net Production Difference Four']].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])

In [ ]:
idx = 360 * 24 * 3
sns.lineplot(
    data=df[[
        "Net Production",
        # "Net Production Estimated One",
        # "Net Production Estimated Two",
        # "Net Production Estimated Three",
        "Net Production Estimated Four",
        # "Net Production Estimated Five",
    ]].iloc[idx:idx + 14*24],
    # dashes=False
)

In [ ]:
sns.boxenplot(
    data=df[[
        "Net Production",
        # "Net Production Estimated One",
        # "Net Production Estimated Two",
        # "Net Production Estimated Three",
        "Net Production Estimated Four",
        # "Net Production Estimated Five",
    ]].rename(columns={
        "Net Production": "Actual Net Production",
        # "Net Production Estimated": "Estimated Net Production",
        "Net Production Estimated Four": "Estimated Net Production",
        # "Net Production Estimated Five": "Estimated Net Production",
    }),
    orient="h",
    showfliers=False
)
plt.xlabel("MWh/h")
plt.show()

In [ ]:
plt.figure(figsize=(15, 3))
sns.histplot(
    df[[
        "Net Production",
        # "Net Production Estimated One",
        # "Net Production Estimated Two",
        # "Net Production Estimated Three",
        "Net Production Estimated Four",
        # "Net Production Estimated Five",
    ]].replace(0, np.nan).rename(columns={
        "Net Production": "Actual Net Production",
        "Net Production Estimated Four": "Estimated Net Production"
        # "Net Production Estimated Five": "Estimated Net Production"
    }),
    kde=True
)
plt.xlabel("MWh/h")
plt.xlim(-700, 600)
plt.show()

# Store Data (CSV)

In [ ]:
# convert index to UTC
df.index = df.index.tz_convert("UTC")
df[[f"Net Production Estimated Four"]].to_csv(f"../data/{plant_name.lower().replace(" @", "_")}_pumping.csv")